## PDS Group 7

In [2]:
import pandas as pd
import numpy as np

pd.set_option("mode.copy_on_write", True)

In [5]:
# Load the processed ARCOs data and display its info
arcos = pd.read_parquet("arcos_processed.parquet")
arcos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162771 entries, 0 to 162770
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   BUYER_STATE   162771 non-null  object 
 1   BUYER_COUNTY  162771 non-null  object 
 2   YEAR          162771 non-null  int32  
 3   TOTAL_MME     162771 non-null  float64
dtypes: float64(1), int32(1), object(2)
memory usage: 4.3+ MB


In [6]:
# check for missing values in each column
arcos.isna().sum()

BUYER_STATE     0
BUYER_COUNTY    0
YEAR            0
TOTAL_MME       0
dtype: int64

In [7]:
arcos.sample(15)

,BUYER_STATE,BUYER_COUNTY,YEAR,TOTAL_MME
83249,GA,PAULDING,2009,2489.119281
162310,ID,MINIDOKA,2013,2143.965365
124371,NC,DAVIE,2011,860.531010
105103,GA,MILLER,2012,177.237779
33891,FL,CLAY,2006,1381.372997
132994,NC,CHOWAN,2007,252.926837
124365,NC,DAVIDSON,2014,4438.218976
159872,OR,COOS,2012,7628.776635
154255,SC,CHARLESTON,2010,55916.277146
79897,GA,CRAWFORD,2007,83.619071


In [16]:
arcos["merge_on"] = (
    arcos["BUYER_COUNTY"].str.upper() + " COUNTY, " + arcos["BUYER_STATE"]
)

In [17]:
arcos["merge_on"].sample(10)

124756          PITT COUNTY, NC
12005      ALLENDALE COUNTY, SC
142        EFFINGHAM COUNTY, GA
8231           DIXIE COUNTY, FL
20755         BENTON COUNTY, WA
147295      OKANOGAN COUNTY, WA
79880           COOK COUNTY, GA
140587      THURSTON COUNTY, WA
52233     OKEECHOBEE COUNTY, FL
78545          CRISP COUNTY, GA
Name: merge_on, dtype: object

In [3]:
pop = pd.read_csv("merged_mortality_population.csv")
pop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61559 entries, 0 to 61558
Data columns (total 13 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   County                           61559 non-null  object
 1   County Code                      61559 non-null  int64 
 2   Year                             61559 non-null  int64 
 3   Drug/Alcohol Induced Cause       61559 non-null  object
 4   Drug/Alcohol Induced Cause Code  61559 non-null  object
 5   Deaths                           61559 non-null  int64 
 6   STATE                            61559 non-null  int64 
 7   COUNTY                           61559 non-null  int64 
 8   STNAME                           61559 non-null  object
 9   CTYNAME                          61559 non-null  object
 10  year                             61559 non-null  int64 
 11  population                       61559 non-null  int64 
 12  fips                            

In [4]:
pop.sample(10)

,County,County Code,Year,Drug/Alcohol Induced Cause,Drug/Alcohol Induced Cause Code,Deaths,STATE,COUNTY,STNAME,CTYNAME,year,population,fips
7264,"Clark County, SD",46025,2004,All other non-drug and non-alcohol causes,O9,44,46,25,South Dakota,Clark County,2004,3897,46025
16214,"Sanpete County, UT",49039,2006,All other non-drug and non-alcohol causes,O9,164,49,39,Utah,Sanpete County,2006,25051,49039
24836,"Yoakum County, TX",48501,2008,All other non-drug and non-alcohol causes,O9,58,48,501,Texas,Yoakum County,2008,7765,48501
39186,"Monroe County, FL",12087,2011,All other non-drug and non-alcohol causes,O9,602,12,87,Florida,Monroe County,2011,73967,12087
3696,"Clarke County, VA",51043,2003,All other non-drug and non-alcohol causes,O9,134,51,43,Virginia,Clarke County,2003,13110,51043
19187,"Erie County, NY",36029,2007,All other alcohol-induced causes,A9,70,36,29,New York,Erie County,2007,921887,36029
5179,"Ohio County, IN",18115,2004,All other non-drug and non-alcohol causes,O9,64,18,115,Indiana,Ohio County,2004,5868,18115
36898,"Union County, TN",47173,2010,All other non-drug and non-alcohol causes,O9,161,47,173,Tennessee,Union County,2010,19116,47173
4671,"Miami-Dade County, FL",12086,2004,All other non-drug and non-alcohol causes,O9,18124,12,86,Florida,Miami-Dade County,2004,2358684,12086
36956,"Bexar County, TX",48029,2010,Drug poisonings (overdose) Suicide (X60-X64),D2,18,48,29,Texas,Bexar County,2010,1722838,48029


In [21]:
merged = arcos.merge(
    pop, how="inner", right_on=["County", "year"], left_on=["merge_on", "YEAR"]
)

In [23]:
merged

,BUYER_STATE,BUYER_COUNTY,YEAR,TOTAL_MME,merge_on,County,County Code,Year,Drug/Alcohol Induced Cause,Drug/Alcohol Induced Cause Code,Deaths,STATE,COUNTY,STNAME,CTYNAME,year,population,fips
